In [1]:
# import Pkg; Pkg.add("CorpusLoaders")

In [1]:
using Knet, Random, Plots, IterTools
using Statistics: mean
using Base.Iterators: flatten
using Embeddings

In [2]:
const w2v = load_embeddings(Word2Vec)

Embeddings.EmbeddingTable{Array{Float32,2},Array{String,1}}(Float32[0.06731994 0.05295623 … -0.21142985 0.013637338; -0.05344657 0.06545979 … -0.0087888 -0.07428761; … ; -0.0073346887 0.010894641 … -0.0040515745 0.015611163; -0.0051456536 -0.047072206 … -0.034157887 0.039655942], ["</s>", "in", "for", "that", "is", "on", "##", "The", "with", "said"  …  "#-###-PA-PARKS", "Lackmeyer", "PERVEZ", "KUNDI", "Budhadeb", "Nautsch", "Antuane", "tricorne", "VISIONPAD", "RAFFAELE"])

In [3]:
S = 30 # max sentence length
# V = length(w2v.vocab) # vocab size
B = 50 # batch size
D = 100 # embedding size
C = 6 # number of classes
Ci = 1 # number of input channels
Ks = [3,4,5] # kernel_sizes
Co = 100 # output channels per kernel size
dp = 0.5; # dropout probability

In [4]:
const get_word_index = Dict(word=>ii for (ii,word) in enumerate(w2v.vocab));

# function get_embedding(word)
#     ind = get_word_index[word]
#     emb = w2v.embeddings[:,ind]
#     return emb
# end

In [5]:
yw2i = Dict{String,Int}()
unk = get_word_index["UNK"]
pad = get_word_index["PAD"]

31875

In [8]:
function readdata(trnf, tstf, S)
    ixtrn = []
    ixtst = []
    
    # read lines
    trnlines = readlines(trnf)
    
    # split label from sentence 
    ytrn = [split(i)[1] for i in trnlines]
    ytrn = [split(i, ":")[1] for i in ytrn]
    xtrn = [split(i)[2:end] for i in trnlines]
    
    # encode label
    ytrn = [get!(yw2i, w, 1+length(yw2i)) for w in ytrn]
    
    
    tstlines = readlines(tstf)
    ytst = [split(i)[1] for i in tstlines]
    ytst = [split(i, ":")[1] for i in ytst]
    ytst = [get(yw2i, w, 0) for w in ytst]
    xtst = [split(i)[2:end] for i in tstlines]
    
    # encode sentences and pad or truncate
    for (i, line) in enumerate(xtrn)
        words = [get(get_word_index, w, unk) for w in line]
        # pad or truncate
        if length(words) >= S
            words = words[1:S]
        else
            words = [words; repeat([pad], S-length(words))]
        end
        push!(ixtrn, words)
    end
    
    for (i, line) in enumerate(xtst)
        words = [get(get_word_index, w, unk) for w in line]
        # pad or truncate
        if length(words) >= S
            words = words[1:S]
        else
            words = [words; repeat([pad], S-length(words))]
        end
        push!(ixtst, words)
    end
    
    # concatenate to one big matrix
    ixtrn, ixtst = hcat(ixtrn...), hcat(ixtst...)
#     @show size(ixtst)
    
    # reduce the size of the 
    unq = [] # unique words in the training and test sets
    push!(unq, unique(ixtrn)...)
    push!(unq, unique(ixtst)...)
    unq = unique(unq)
    sort!(unq)
#     @show size(unq)
    trans = Dict(j => i for (i,j) in enumerate(unq)) 
    ixtrn = [get(trans, i, 0) for i in ixtrn]
#     @show size(ixtrn)
    ixtst = [get(trans, i, 0) for i in ixtst]
    embeddings = KnetArray(w2v.embeddings[:,unq])
    global V = size(embeddings, 2)
    # create iterators
    dtrn, dtst = minibatch(ixtrn, ytrn, B, shuffle=true), minibatch(ixtst, ytst, B, shuffle=true)
    return dtrn, dtst, embeddings
end

readdata (generic function with 1 method)

In [9]:
dtrn, dtst, embedding = readdata("./TREC/train_5500.label.txt", "./TREC/TREC_10.label.txt", S)

size(unq) = (8970,)
size(ixtrn) = (30, 5452)


(Knet.Data{Tuple{Array{Int64,2},Array{Int64,1}}}([988 423 … 423 423; 88 1788 … 4 1699; … ; 6647 6647 … 6647 6647; 6647 6647 … 6647 6647], [1 2 … 5 2], 50, 5452, false, 5403, 1:5452, true, (30, 5452), (5452,), Array{Int64,2}, Array{Int64,1}), Knet.Data{Tuple{Array{Int64,2},Array{Int64,1}}}([988 423 … 423 423; 313 787 … 4 4; … ; 6647 6647 … 6647 6647; 6647 6647 … 6647 6647], [5 6 … 2 1], 50, 500, false, 451, 1:500, true, (30, 500), (500,), Array{Int64,2}, Array{Int64,1}), K32(300,8970)[0.05295623⋯])

In [60]:
# Embed layer # may load pretrained embeddings instead
struct Embed; w; end
(l::Embed)(x) = reshape(l.w[:,x], (size(l.w,1),size(x,1),1,size(x,2))) # E,S,Cx,B

In [61]:
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [62]:
struct Conv; w; b; f; p; end
function (c::Conv)(x)
    conved = conv4(c.w, dropout(x,c.p)) .+ c.b
    return c.f.(pool(conved; window=(size(conved, 1), size(conved, 2))))
end
Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Conv

In [63]:
struct cnntext
    V # vocab size
    D # embedding size
    C # number of classes
    Ci # one
    Ks # kernel_sizes
    Co # number of each of the sizes
    dp # dropout probability
    embed::Embed # embedding layer
    fc::Dense 
    conv1::Conv
    conv2::Conv
    conv3::Conv
end

function (c::cnntext)(x)
    x = c.embed(x)  # E, S, Cx (1), B
    x1, x2, x3 = c.conv1(x), c.conv2(x), c.conv3(x) # 
    x = cat(x1,x2,x3, dims=1)
    x = c.fc(x)
end

(c::cnntext)(x,y) = nll(c(x),y)

(c::cnntext)(d::Knet.Data) = mean(c(x,y) for (x,y) in d)

function cnntext(V, D, C, Ci, Ks, Co, dp)  ##############
    cnntext(V, D, C, Ci, Ks, Co, dp, Embed(Param(embedding)), Dense(length(Ks)*Co,C,identity,pdrop=dp), Conv(D,Ks[1],Ci,Co), Conv(D,Ks[3],Ci,Co), Conv(D,Ks[3],Ci,Co))
end

cnntext

In [64]:
model = cnntext(V, D, C, Ci, Ks, Co, dp)

cnntext(8970, 100, 6, 1, [3, 4, 5], 100, 0.5, Embed(P(KnetArray{Float32,2}(300,8970))), Dense(P(KnetArray{Float32,2}(6,300)), P(KnetArray{Float32,1}(6)), identity, 0.5), Conv(P(KnetArray{Float32,4}(100,3,1,100)), P(KnetArray{Float32,4}(1,1,100,1)), NNlib.relu, 0), Conv(P(KnetArray{Float32,4}(100,5,1,100)), P(KnetArray{Float32,4}(1,1,100,1)), NNlib.relu, 0), Conv(P(KnetArray{Float32,4}(100,5,1,100)), P(KnetArray{Float32,4}(1,1,100,1)), NNlib.relu, 0))

In [65]:
first(dtrn)[1]

30×50 Array{Int64,2}:
  423   423    67   988   423   423  …   423  3150   423  1935   423   423
    9     4  7906   124    55     4     1895   831     4     4   184     4
 2938  1895  1047  4426    10  7906     5198    88  4147  6529     1    10
 2594  7906  7906   169  7906   500     2706  8699  5289  5607  7906  3732
 7906  6647    80    25  5507  5214        4  7562  7906  2136    33  7906
  666  6647   169  6350     5     1  …  1251  1359   765  3108    10    10
 1380  6647  7900    20  7906  2856     2636  7906  7906  1383   101   435
  502  6647   956  7906  7109  3563       10  6647  2178  5521  5396  5199
 7906  6647  7906  6647  3160  7906     1892  6647  7906    65  2665  6163
 6647  6647  6647  6647   978  6647      591  6647  6647   430  7906  7906
 6647  6647  6647  6647     2  6647  …  7906  6647  6647     1  1118  6647
 6647  6647  6647  6647  7906  6647     6647  6647  6647  7906  1080  6647
 6647  6647  6647  6647  6647  6647     6647  6647  6647  6647  7906  6647
   

In [66]:
model(first(dtrn)[1])

6×50 KnetArray{Float32,2}:
  0.0251474    0.029747     0.0268792   …   0.0295141    0.0232194 
  0.00715772   0.00108968   0.00335438      0.00522249   0.00180868
  0.0681352    0.0621302    0.0648593       0.065296     0.0695555 
 -0.0168696   -0.0229652   -0.0131215      -0.0155905   -0.00671204
  0.028964     0.0292695    0.0241421       0.0234454    0.0334205 
 -0.0276406   -0.0244257   -0.0299898   …  -0.026911    -0.0293199 

In [67]:
nepochs = 2

2

In [68]:
function trainresults(file,model; o...)
    if (print("Train from scratch? "); readline()[1]=='y')
        r = ((model(dtrn), model(dtst), accuracy(model,dtrn), accuracy(model,dtst))
             for x in takenth(progress(adam(model,ncycle(dtrn,nepochs))),length(dtrn)))
        r = reshape(collect(Float32,flatten(r)),(4,:))
        Knet.save(file,"results",r)
        Knet.gc() # To save gpu memory
    else
        isfile(file) || return
        r = Knet.load(file,"results")
    end
    println(minimum(r,dims=2))
    return r
end

trainresults (generic function with 1 method)

In [69]:
cnn = trainresults("cnntextTREC.jld2", model);

Train from scratch? stdin> y


┣███████████████████▉┫ [100.00%, 218/218, 09:09/09:09, 2.52s/i] 


Float32[0.38689047; 0.4562152; 0.6981651; 0.712]


In [70]:
cnn

4×2 Array{Float32,2}:
 0.918583  0.38689 
 1.0056    0.456215
 0.698165  0.899266
 0.712     0.86    

In [23]:
# This program has requested access to the data dependency word2vec 300d.
# which is not currently installed. It can be installed automatically, and you will not see this message again.

# Pretrained Word2Vec Word emeddings
# Website: https://code.google.com/archive/p/word2vec/
# Author: Mikolov et al.
# Year: 2013

# Pre-trained vectors trained on part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases.

# Paper:
#     Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado, and Jeffrey Dean. Distributed Representations of Words and Phrases and their Compositionality. In Proceedings of NIPS, 2013.



# Do you want to download the dataset from https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz to "/home/saif/.julia/datadeps/word2vec 300d"?
# [y/n]
# stdin> y
# const w2v = load_embeddings(Word2Vec)

In [7]:
# l = length.(xtrn)

# using StatsBase;
# a = countmap(l);

# b = hcat([[key, val] for (key, val) in a]...)';

# sort(collect(a), by=x->x[2]);